In [1]:
!pip install google
!pip install transformers
!pip install bert-extractive-summarizer
!pip install newspaper3k

In [2]:
from newspaper import fulltext
from summarizer import Summarizer
from googlesearch import search 
from transformers import *
import requests
sentiment = pipeline('sentiment-analysis')
abst = TFAutoModelWithLMHead.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

/usr/local/lib/python3.6/dist-packages/transformers/models/auto/modeling_tf_auto.py:776: FutureWarning: The class `TFAutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `TFAutoModelForCausalLM` for causal language models, `TFAutoModelForMaskedLM` for masked language models and `TFAutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [3]:
import re
article_url="https://analyticsindiamag.com/is-common-sense-common-in-nlp-models/ "
article = fulltext(requests.get(article_url).text)
article=''.join([i for i in article.splitlines() if i])
article

'NLP Models have shown tremendous advancements in syntactic, semantic and linguistic knowledge for downstream tasks. However, that raises an interesting research question — is it possible for them to go beyond pattern recognition and apply common sense for word-sense disambiguation?Thus, to identify if BERT, a large pre-trained NLP model developed by Google, can solve common sense tasks, researchers took a closer look. The researchers from Westlake University and Fudan University, in collaboration with Microsoft Research Asia, discovered how the model computes the structured, common sense knowledge for downstream NLP tasks.According to the researchers, it has been a long-standing debate as to whether pre-trained language models can solve tasks leveraging only a few shallow clues and their common sense of knowledge. To figure that out, researchers used a CommonsenseQA dataset for BERT to solve multiple-choice problems.In this research, the analysts used CONCEPTNET focusing on attention 

In [9]:
def get_summary(article_url,choice):
  article = fulltext(requests.get(article_url).text)
  article=''.join([i for i in article.splitlines() if i])
  if (choice):
    model = Summarizer()
    result = model(article, min_length=40)
    summary = ''.join(result)  
  else:#0 will return this
    inputs = tokenizer.encode("summarize: " + article, return_tensors="tf", max_length=512,truncation=True)
    summary = abst.generate(inputs, max_length=250, min_length=40, length_penalty=1.5, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary[0])
  return summary

In [4]:
def get_url(query):
  urls = []
  for j in search(query, tld="com", num=10, stop=10, pause=2): 
      if j.count('wiki')==0:
          urls.append(j)
  return urls
urls=get_url("Farmer bill protest")
print(len(urls))

9


In [6]:
ext_data={"Summary":[],"Source":[],"SS":[],"Label":[]}

for i in urls:
  summary=get_summary(i,1)
  s=sentiment(summary)
  print(f"summary\n{summary}\nby = {i}")
  ext_data['Summary'].append(summary)
  ext_data['Source'].append(i)
  ext_data['SS'].append(s[0]['label'])
  ext_data['Label'].append(s[0]['score'])
  print('*'*50)



summary
At the protest Singhu border where farmers are protesting against the farm laws, in New Delhi on Tuesday. ( BKU (Dakuanda) general secretary Jagmohan Singh Patiala explains point-by-point why farmers are objecting to Centre’s offer. So, the Centre is not ready to take any responsibility on this issue. This is also being proposed under the pressures of farmer organisations. In such cases farmers are unable to repay the loans and have no option other than to sell/lose their lands. Even in the USA, where huge subsidies are given for the agriculture sector the farmers are forced to commit suicides.” Therefore the Modi government wants to bring the power sector under the central control. ”Centre’s proposal: Willing to amend law on controlling NCR pollutionFarmers’ objection: “Union government thinks that stubble burning of paddy is responsible for pollution in Delhi and so they have enacted a legislation of stringent punishments and heavy fines up to Rs 1 crore for the alleged viol

In [13]:
import pandas as pd
pd.DataFrame.from_dict(ext_data).to_csv("ext_summary.csv")

**<h2> Abstractive </h2>**

In [12]:
abstractive_data={"Summary":[],"Source":[],"SS":[],"Label":[]}
for i in urls:
    summary = get_summary(i,0)
    s = sentiment(summary)
    print(f"summary\n{summary}\nby = {i}\nsentiment = {s}")
    abstractive_data['Summary'].append(summary)
    abstractive_data['Source'].append(i)
    abstractive_data['SS'].append(s[0]['label'])
    abstractive_data['Label'].append(s[0]['score'])
    print('*'*50)

summary
<pad> farmer unions lost no time in rejecting the amendments proposed by the Union government to the current farm laws. creation of private mandis along with the state-run agriculture produce market committees (APMC) will push all agriculture businesses towards private markets.
by = https://indianexpress.com/article/explained/government-farmer-talks-deadlock-explained-7106698/
sentiment = [{'label': 'POSITIVE', 'score': 0.9189099669456482}]
**************************************************
summary
<pad> farmers from Punjab and Haryana have laid siege to the national capital. they are protesting against recently passed Farm Bills. farmers fear that this may be an excuse to pull off the MSP safety net.
by = https://www.timesnownews.com/india/article/what-is-the-farm-bill-and-why-are-farmers-protesting-against-it/689215
sentiment = [{'label': 'NEGATIVE', 'score': 0.9960730075836182}]
**************************************************
summary
<pad> the Act on Agri market allows fa